# Airbyte Stripe (已弃用)

请注意：此连接器特定加载器已弃用。请改用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个数据集成平台，用于从 API、数据库和文件到数据仓库和数据湖的 ELT 管道。它拥有最大的到数据仓库和数据库的 ELT 连接器目录。

此加载器将 Stripe 连接器公开为一个文档加载器，允许您将各种 Stripe 对象加载为文档。

## 安装

首先，你需要安装 `airbyte-source-stripe` Python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-stripe

## 示例

请查阅 [Airbyte 文档页面](https://docs.airbyte.com/integrations/sources/stripe/)，了解如何配置读取器。

配置对象应遵循的 JSON schema 可以在 Github 上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-stripe/source_stripe/spec.yaml](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-stripe/source_stripe/spec.yaml)。

总体结构如下：
```python
{
  "client_secret": "<secret key>",
  "account_id": "<account id>",
  "start_date": "<date from which to start retrieving records from in ISO format, e.g. 2020-10-20T00:00:00Z>",
}
```

默认情况下，所有字段都将作为元数据存储在文档中，文本内容为空字符串。通过转换读取器返回的文档来构建文档的文本内容。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteStripeLoader

config = {
    # your stripe configuration
}

loader = AirbyteStripeLoader(
    config=config, stream_name="invoices"
)  # check the documentation linked above for a list of all streams

您现在可以通过常规方式加载文档了

In [ ]:
docs = loader.load()

由于 `load` 返回一个列表，它将一直阻塞直到所有文档都加载完成。为了更好地控制此过程，您还可以使用 `lazy_load` 方法，该方法返回一个迭代器：

In [ ]:
docs_iterator = loader.lazy_load()

请注意，默认情况下，页面内容为空，并且 metadata 对象包含记录中的所有信息。要以其他方式创建文档，请在创建加载器时传入一个 `record_handler` 函数：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteStripeLoader(
    config=config, record_handler=handle_record, stream_name="invoices"
)
docs = loader.load()

## 增量加载

某些数据流支持增量加载，这意味着源会跟踪已同步的记录，并且不会再次加载它们。这对于数据量大且频繁更新的源非常有用。

要利用此功能，请存储 loader 的 `last_state` 属性，并在再次创建 loader 时将其传入。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteStripeLoader(
    config=config,
    record_handler=handle_record,
    stream_name="invoices",
    state=last_state,
)

new_docs = incremental_loader.load()